In [1]:
import pandas as pd
import numpy as np
import torch
import warnings

# Define file paths
ais_tracks_path = '../../data/tracks_ais.csv'
radar_detections_path = '../../data/detections_radar.csv'

ais_tracks = pd.read_csv(ais_tracks_path)
radar_detections = pd.read_csv(radar_detections_path)

/Users/liuzehan/miniconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
BATCH_SIZE = 32
MAX_LENGTH = 256
HIDDEN_DIM = 64
NUM_LAYERS_GRU = 1


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


## Preprocess

In [10]:
from utilities import VesselTypeAggregator, haversine_distance, SumStats

# Preprocessing: add `datetime`, `time_elapsed`
def preprocess_group(group):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        group['datetime'] = pd.to_datetime(group['cdate'] + ' ' + group['ctime'])
        group = group.sort_values('datetime')  # Ensure chronological order
        group['time_diff'] = group['datetime'].diff().dt.total_seconds().fillna(0)
        group['time_elapsed'] = group['time_diff'].cumsum()

        start_latitude = group['latitude'].iloc[0]
        start_longitude = group['longitude'].iloc[0]
        group['distance_from_origin'] = haversine_distance(group['latitude'], group['longitude'], start_latitude, start_longitude)
        group['latitude_prev'] = group['latitude'].shift(1)
        group['longitude_prev'] = group['longitude'].shift(1)
        group['distance_travelled'] = haversine_distance(group['latitude_prev'], group['longitude_prev'], group['latitude'], group['longitude'])
        group['distance_travelled'] = group['distance_travelled'].fillna(0)
        group['instant_speed'] = group['distance_travelled'] / group['time_diff'] * 1943 # to knots

        group['disrupted'] = False
        if group['instant_speed'].max() >= 150:
            group['disrupted'] = True
    return group

# Get Labels from AIS
merged_tracks = pd.merge(
    ais_tracks[['assoc_id', 'type_m2']],
    radar_detections,
    left_on='assoc_id',
    right_on='id_track',
    how='inner'
)

#Preprocess
#Removes disrupted tracks (~8% removed)
merged_tracks = merged_tracks.groupby('id_track').apply(preprocess_group).reset_index(drop=True)
merged_tracks = merged_tracks[merged_tracks['disrupted'] == False]

#summary vector calculation
sum_stats_df = merged_tracks.groupby('id_track').apply(SumStats.compute_track_stats).reset_index()
summary_cols = ['avg_speed', 'max_speed', 'min_speed', 'total_distance', 'curviness', 'heading_std', 'turning_mean', 'turning_std']
summary_lookup = sum_stats_df.set_index('id_track')[summary_cols].to_dict(orient='index')

#Aggregate vessel type to acquire target label: type_m2_agg
vessel_type_aggregator = VesselTypeAggregator()
vessel_type_aggregator.aggregate_vessel_type(merged_tracks)

# Feature and label columns
feature_cols = ['speed', 'course', 'time_elapsed', 'distance_travelled', 'distance_from_origin']
label_col = 'type_m2_agg'

# Build lookup dictionary
label_classes = {label: i for i, label in enumerate(merged_tracks[label_col].unique())}
type_dict = merged_tracks.drop_duplicates('id_track').set_index('id_track')[label_col].map(label_classes).to_dict()

# Group by track
grouped = merged_tracks.groupby('id_track')

# Initialize Dataset
track_data = []
for id_track, group in grouped:
    if id_track not in type_dict:
        continue

    features = torch.tensor(group[feature_cols].values, dtype=torch.float32)
    length = features.size(0)
    summary_vector = torch.tensor(list(summary_lookup[id_track].values()), dtype=torch.float32)
    track_data.append({
        'features': features.to(device),  # T x M
        'summary': summary_vector.to(device),
        'length': length,
        'label': torch.tensor(type_dict[id_track], dtype=torch.long).to(device)
    })

print(f"Prepared {len(track_data)} track tensors (raw features) on {device}")

/var/folders/jm/bv3tgs257hx9tx764_p9xq0w0000gn/T/ipykernel_19509/1060896074.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_tracks = merged_tracks.groupby('id_track').apply(preprocess_group).reset_index(drop=True)
/var/folders/jm/bv3tgs257hx9tx764_p9xq0w0000gn/T/ipykernel_19509/1060896074.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sum_stats_df = merged_tracks.groupby('id_track').apply(

Prepared 14366 track tensors (raw features) on mps


In [12]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Sort by sequence length (optional but helpful for some RNNs)
    batch.sort(key=lambda x: x['length'], reverse=True)
    features = [item['features'] for item in batch]
    summaries = torch.stack([item['summary'] for item in batch])
    labels = torch.stack([item['label'] for item in batch])

    labels = torch.stack([item['label'] for item in batch])
    lengths = torch.tensor([seq.size(0) for seq in features])
    padded_features = pad_sequence(features, batch_first=True)  # B, T_max, M

    return padded_features, lengths, summaries, labels

In [13]:
from torch.utils.data import random_split, DataLoader, Dataset

class VesselDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Set random seed for reproducibility
seed = 42
torch.manual_seed(seed)

# Total size
total_size = len(track_data)

# Sizes for split
test_size = val_size = int(0.15 * total_size)
train_size = total_size - test_size - val_size

# Perform the split
full_dataset = VesselDataset(track_data)
train_set, val_set, test_set = random_split(full_dataset, [train_size, val_size, test_size])

# Sample 20% of the training set for lightweight training
small_train_size = int(0.4 * len(train_set))
indices = np.random.choice(len(train_set), small_train_size, replace=False)
small_train_subset = torch.utils.data.Subset(train_set, indices)

# DataLoaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

print(f"Data split: {len(train_set)} train, {len(val_set)} val, {len(test_set)} test")


Data split: 10058 train, 2154 val, 2154 test


In [14]:
import torch.nn as nn

class VesselRNNClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_layers, summary_dim):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim + summary_dim, num_classes)

    def forward(self, x, lengths, summaries):
        #lengths: length B, indicating real length of each sequence
        x_padded = nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False) # N * T_max * M
        _, hidden = self.gru(x_padded) 

        concat_layer = torch.cat((hidden[-1], summaries), dim=1)

        #concatenate hidden state with summary vector
        logits = self.fc1(concat_layer)  # B * num_classes
        return logits


In [16]:
EPOCH = 5

from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

input_dim = len(feature_cols)
num_classes = len(label_classes)

model = VesselRNNClassifier(input_dim, HIDDEN_DIM, num_classes, num_layers=NUM_LAYERS_GRU, summary_dim=len(summary_cols)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
scheduler = StepLR(optimizer, step_size=2, gamma=0.5) 

train_losses = []
val_losses = []

for epoch in range(EPOCH):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

    for x_batch, lengths, summaries, y_batch in progress_bar:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        summaries = summaries.to(device)
        
        optimizer.zero_grad()
        logits = model(x_batch, lengths, summaries) #B * num_classes
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Evaluate on validation set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x_val, lengths_val, summaries_val, y_val in val_loader:
            output_val = model(x_val, lengths_val, summaries_val)
            loss_val = criterion(output_val, y_val)
            val_loss += loss_val.item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"✅ Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

save_path = 'models/gru_5_epochs.pth'

torch.save({
    'epoch': epoch + 1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'input_dim': input_dim,
    'hidden_dim': HIDDEN_DIM,
    'num_classes': num_classes,
}, save_path)


✅ Epoch 1 | Train Loss: 1.8236 | Val Loss: 1.8552


✅ Epoch 2 | Train Loss: 1.7629 | Val Loss: 2.0339


✅ Epoch 3 | Train Loss: 1.4894 | Val Loss: 1.3750


✅ Epoch 4 | Train Loss: 1.4962 | Val Loss: 1.5347


✅ Epoch 5 | Train Loss: 1.3493 | Val Loss: 1.4799


### Resume Training

In [ ]:
##TODO: UPDATE MODEL 

checkpoint = torch.load('models/gru_5_epochs.pth', map_location=device)

# Recreate the model architecture
model = VesselRNNClassifier(
    input_dim=checkpoint['input_dim'],
    hidden_dim=checkpoint['hidden_dim'],
    num_classes=checkpoint['num_classes'],
    num_layers=NUM_LAYERS_GRU
).to(device)

# Load saved state
model.load_state_dict(checkpoint['model_state_dict'])

# Restore optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)  # same LR as before
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Continue tracking losses
TRAIN_LOSSES = checkpoint['train_losses']
VAL_LOSSES = checkpoint['val_losses']

# Resume from this epoch
start_epoch = checkpoint['epoch']

for epoch in range(start_epoch, start_epoch + 5):  # continue 5 more epochs
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False)

    for x_batch, lengths, y_batch in progress_bar:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        logits = model(x_batch, lengths)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    scheduler.step()

    avg_train_loss = total_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Evaluate on validation set
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x_val, lengths_val, y_val in val_loader:
            output_val = model(x_val, lengths_val)
            loss_val = criterion(output_val, y_val)
            val_loss += loss_val.item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"✅ Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


✅ Epoch 6 | Train Loss: 1.6352 | Val Loss: 1.6514


✅ Epoch 7 | Train Loss: 1.6380 | Val Loss: 1.6324


✅ Epoch 8 | Train Loss: 1.6363 | Val Loss: 1.6632


✅ Epoch 9 | Train Loss: 1.6342 | Val Loss: 1.6847


✅ Epoch 10 | Train Loss: 1.6393 | Val Loss: 1.6743


...

✅ Epoch 36 | Train Loss: 1.6207 | Val Loss: 1.6785


✅ Epoch 37 | Train Loss: 1.6098 | Val Loss: 1.6317


✅ Epoch 38 | Train Loss: 1.6064 | Val Loss: 1.5775


✅ Epoch 39 | Train Loss: 1.5869 | Val Loss: 1.6311


✅ Epoch 40 | Train Loss: 1.5973 | Val Loss: 1.6602


In [55]:
##TODO: UPDATE MODEL PATH

model_path = 'models/gru_40_epochs.pth'

torch.save({
    'epoch': epoch + 1,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'train_losses': train_losses,
    'val_losses': val_losses,
    'input_dim': input_dim,
    'hidden_dim': HIDDEN_DIM,
    'num_classes': num_classes,
}, model_path)


### Model Eval

In [18]:
# Ensure model is in eval mode
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():

    progress_bar = tqdm(test_loader, desc=f"Running Test", leave=False)
    for x_test, lengths_test, summaries_test, y_test in progress_bar:
        x_test = x_test.to(device)
        lengths_test = lengths_test  # assuming these are CPU-side already
        logits = model(x_test, lengths_test, summaries_test)

        preds = torch.argmax(logits, dim=1)  # get class with highest logit
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_test.cpu().numpy())  # true labels

# Convert to numpy arrays if needed
import numpy as np
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

# (Optional) Evaluate accuracy or confusion matrix
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("🔍 Test Accuracy:", accuracy_score(all_labels, all_preds))
print("\n📊 Classification Report:\n", classification_report(all_labels, all_preds))


🔍 Test Accuracy: 0.5371402042711235

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.23      0.32       158
           1       0.00      0.00      0.00        25
           2       0.67      0.92      0.78       869
           3       0.74      0.05      0.10       387
           4       0.25      0.05      0.09        39
           5       0.35      0.78      0.48       376
           6       0.18      0.01      0.02       224
           7       0.00      0.00      0.00        76

    accuracy                           0.54      2154
   macro avg       0.34      0.26      0.22      2154
weighted avg       0.52      0.54      0.44      2154



/Users/liuzehan/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/liuzehan/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/liuzehan/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
